<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Deep-Learning/blob/master/IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [2]:
import tensorflow_datasets as tfds

imdb, inf0 = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5U6N6R/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5U6N6R/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5U6N6R/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_data, test_data = imdb['train'], imdb['test']

In [0]:
import numpy as np

training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [0]:
vocab_size = 10000
embedding_dim = 12
max_length = 120
oov_tok = '<OOV>'
trunc_type = 'post'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)




In [9]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(key, '?') for key in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b'this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received '
b'This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.'


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 12)           120000    
_________________________________________________________________
flatten (Flatten)            (None, 1440)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 8646      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 128,653
Trainable params: 128,653
Non-trainable params: 0
_________________________________________________________________


In [11]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5086 - accuracy: 0.7309 - val_loss: 0.3496 - val_accuracy: 0.8438
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2488 - accuracy: 0.9013 - val_loss: 0.3633 - val_accuracy: 0.8417
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1048 - accuracy: 0.9716 - val_loss: 0.4356 - val_accuracy: 0.8307
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0298 - accuracy: 0.9956 - val_loss: 0.5243 - val_accuracy: 0.8244
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0078 - accuracy: 0.9994 - val_loss: 0.5923 - val_accuracy: 0.8257
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.6460 - val_accuracy: 0.8271
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.6917 - val_accuracy: 0.8269
Epoch 

In [15]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(10000, 12)
[[-0.01483684  0.02305467 -0.04553576 ... -0.02119227 -0.01218429
   0.0585877 ]
 [ 0.03630466 -0.05932578 -0.02281269 ... -0.01326701  0.0231772
   0.06896508]
 [ 0.01332932 -0.00711497 -0.01201109 ... -0.03711967 -0.0215215
   0.14827794]
 ...
 [ 0.06959699  0.04640483  0.03666154 ...  0.0050758  -0.02915708
   0.13304123]
 [ 0.01102282  0.07726528 -0.07853525 ... -0.01133747 -0.07920197
  -0.0978649 ]
 [-0.06474859  0.13198069  0.0573907  ...  0.08814909  0.06319998
   0.07816216]]


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [18]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1431], [966], [4], [1537], [1537], [4715], [], [790], [2019], [11], [2929], [2184], [], [790], [2019], [11], [579], [], [11], [579], [], [4], [1782], [4], [4517], [11], [2929], [1275], [], [], [2019], [1003], [2929], [966], [579], [790], []]
